In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('21年6月实时数据NJ-S122其林门_train.csv')
df['volume'] = df[['XKC', 'DKC', 'XHC', 'ZHC', 'DHC', 'TDH']].sum(axis=1)
df = df[(df['volume']>0) & (df['PJCTJJ']>0) & (df['SJZYL']>0)]
df = df.reset_index(drop=True)
df.head()

,YEAR,GCRQ,GCZBS,HOUR,MINUTE,CDH,XSFX,SBSBM,DCSJLX,SJXH,...,OFF_MINS,ERR_CODE,ERR_DESC,DELETE_BY,DELETE_TIME,CREATE_BY,CREATE_TIME,UPDATE_BY,UPDATE_TIME,volume
0,2021,24-JUN-21,32000020160926C5B281FC540615BAA6,20,10,33,X,141150316092101,0,231,...,-2.0,NaN,NaN,NaN,NaN,sys,24-JUN-21 07.13.02.757000000 PM,sys,24-JUN-21 07.09.41.672042000 PM,22
1,2021,05-JUN-21,32000020160926C5B281FC540615BAA6,18,0,13,S,141150316092101,0,205,...,-2.1,NaN,NaN,NaN,NaN,sys,05-JUN-21 05.02.57.974000000 PM,sys,05-JUN-21 05.00.32.536633000 PM,38
2,2021,27-JUN-21,32000020160926C5B281FC540615BAA6,6,45,36,X,141150316092101,0,70,...,1.1,NaN,NaN,NaN,NaN,sys,27-JUN-21 05.51.09.066000000 AM,sys,27-JUN-21 05.47.28.190581000 AM,4
3,2021,17-JUN-21,32000020160926C5B281FC540615BAA6,8,40,12,S,141150316092101,0,93,...,2.3,NaN,NaN,NaN,NaN,sys,17-JUN-21 07.47.19.126000000 AM,sys,17-JUN-21 07.58.02.013440000 AM,77
4,2021,04-JUN-21,32000020160926C5B281FC540615BAA6,14,5,12,S,141150316092101,0,158,...,4.9,NaN,NaN,NaN,NaN,sys,04-JUN-21 01.14.57.048000000 PM,sys,04-JUN-21 01.12.22.839632000 PM,57


# KNN

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error


# 过拟合的例子

X = df[['PJCTJJ', 'SJZYL','HOUR','MINUTE', 'CDH', 'OFF_MINS','GCBFB','DHCS']]
y = df['volume']

model_knn = KNeighborsRegressor(n_neighbors=1)
model_knn.fit(X, y)

mean_absolute_error(y, model_knn.predict(X))

0.001841098994476703

In [3]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

for train, val in kf.split(df):
    X_train = df.loc[train, ['PJCTJJ', 'SJZYL','HOUR','MINUTE', 'CDH', 'OFF_MINS','GCBFB','DHCS']]
    y_train = df.loc[train, 'volume']
    X_val = df.loc[val, ['PJCTJJ', 'SJZYL','HOUR','MINUTE', 'CDH', 'OFF_MINS','GCBFB','DHCS']]
    y_val = df.loc[val, 'volume']
    model_knn = KNeighborsRegressor(n_neighbors=1)
    model_knn.fit(X_train, y_train)
    print(mean_absolute_error(y_val, model_knn.predict(X_val)))

4.168824600445074
4.2038235889136155
4.24693980778958
4.223975720789074
4.321598381385939


In [4]:
# 手动调参

def knn(n_neighbors):
    kf = KFold(n_splits=5)
    loss = 0
    for train, val in kf.split(df):
        X_train = df.loc[train, ['PJCTJJ', 'SJZYL','HOUR','MINUTE', 'CDH', 'OFF_MINS','GCBFB','DHCS']]
        y_train = df.loc[train, 'volume']
        X_val = df.loc[val, ['PJCTJJ', 'SJZYL','HOUR','MINUTE', 'CDH', 'OFF_MINS','GCBFB','DHCS']]
        y_val = df.loc[val, 'volume']
        model_knn = KNeighborsRegressor(n_neighbors=n_neighbors)
        model_knn.fit(X_train, y_train)
        loss += mean_absolute_error(y_val, model_knn.predict(X_val))
    return loss / 5

best_loss = 9999999
for i in range(1, 11):
    loss = knn(i)
    print(f'n_neighbors={i}, loss={loss}')
    if loss < best_loss:
        best_i = i
        best_loss = loss
        
print(f'best_i={best_i}, best_loss={best_i}')

n_neighbors=1, loss=4.2330324198646565
n_neighbors=2, loss=3.823043590200926
n_neighbors=3, loss=3.7233512714307464
n_neighbors=4, loss=3.679008579444514
n_neighbors=5, loss=3.6609445978540798
n_neighbors=6, loss=3.6528876523338916
n_neighbors=7, loss=3.65899151592699
n_neighbors=8, loss=3.662087454799586
n_neighbors=9, loss=3.668372972029509
n_neighbors=10, loss=3.673440161083699
best_i=6, best_loss=6


In [5]:
df_test = pd.read_csv('./21年6月实时数据NJ-S122其林门_test.csv')
model = KNeighborsRegressor(n_neighbors=6)
X_train = df.loc[:, ['PJCTJJ', 'SJZYL','HOUR','MINUTE', 'CDH', 'OFF_MINS','GCBFB','DHCS']]
y_train = df.loc[:, 'volume']
model.fit(X_train,y_train)
model.predict(df_test.loc[:, ['PJCTJJ', 'SJZYL','HOUR','MINUTE', 'CDH', 'OFF_MINS','GCBFB','DHCS']])

array([58.83333333, 54.66666667, 18.33333333, ..., 58.33333333,
       24.33333333,  5.33333333])

In [6]:
pd.Series(model.predict(df_test.loc[:, ['PJCTJJ', 'SJZYL','HOUR','MINUTE', 'CDH', 'OFF_MINS','GCBFB','DHCS']])).to_csv('y_pred_knn.csv', index=False)

# SVM

In [7]:
from sklearn.svm import SVR

# 复杂度问题
def svr_with_sample(num):
    samples = df.sample(num).reset_index(drop=True)
    X = samples[['PJCTJJ', 'SJZYL']]
    y = samples['volume']

    model_svr = SVR(kernel='rbf')
    model_svr.fit(X,y)
    return mean_absolute_error(y, model_svr.predict(X))

%timeit svr_with_sample(100)
%timeit svr_with_sample(1000)
%timeit svr_with_sample(2000)

3.18 ms ± 35.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
84 ms ± 643 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
329 ms ± 4.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
from sklearn.model_selection import GridSearchCV

samples = df.sample(1000).reset_index(drop=True)
X = samples[['PJCTJJ', 'SJZYL']]
y = samples['volume']

model_svr = SVR()
param_grid = [
  {'C': [1, 10,], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
clf = GridSearchCV(model_svr, param_grid, scoring='neg_mean_absolute_error', cv=3)
clf.fit(X,y)

GridSearchCV(cv=3, estimator=SVR(),
             param_grid=[{'C': [1, 10], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             scoring='neg_mean_absolute_error')

In [9]:
pd.DataFrame(clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.041003,0.162078,0.004000,4.495664e-07,1,linear,NaN,"{'C': 1, 'kernel': 'linear'}",-7.386701,-6.577736,-7.405147,-7.123195,0.385771,10
1,8.752431,1.389705,0.003667,4.715952e-04,10,linear,NaN,"{'C': 10, 'kernel': 'linear'}",-7.394261,-6.560526,-7.399815,-7.118201,0.394342,9
2,0.014720,0.000513,0.011614,4.389980e-04,1,rbf,0.001,"{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}",-5.551275,-5.397642,-5.768626,-5.572514,0.152196,8
3,0.013904,0.000135,0.011762,5.514297e-04,1,rbf,0.0001,"{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}",-5.199711,-5.230672,-5.343499,-5.257961,0.061792,7
4,0.018001,0.000817,0.011666,4.713704e-04,10,rbf,0.001,"{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}",-4.922879,-4.753034,-4.971945,-4.882620,0.093795,3
5,0.016000,0.000001,0.011333,4.705836e-04,10,rbf,0.0001,"{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}",-4.869274,-4.623052,-5.051741,-4.848022,0.175655,2
6,0.022666,0.001248,0.012334,4.710895e-04,100,rbf,0.001,"{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}",-4.874951,-4.826469,-4.951110,-4.884177,0.051301,4
7,0.019000,0.000817,0.012000,2.973602e-07,100,rbf,0.0001,"{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}",-4.848588,-4.574462,-5.115134,-4.846061,0.220736,1
8,0.077001,0.019096,0.012000,8.165348e-04,1000,rbf,0.001,"{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}",-5.075857,-4.846736,-5.078885,-5.000493,0.108729,6
9,0.040667,0.003858,0.011333,4.714266e-04,1000,rbf,0.0001,"{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}",-4.989848,-4.536446,-5.163734,-4.896676,0.264428,5


In [10]:
y_hat = clf.best_estimator_.predict(df[['PJCTJJ', 'SJZYL']])
mean_absolute_error(df['volume'], y_hat)

4.845640444948096